# 3. Training Pipeline

## 3.1. Setup

### 3.1.1. Import Libraries

In [55]:
# Standard imports
import os
from pathlib import Path
import sys
import json
import time
from datetime import date, datetime, timedelta
import warnings

warnings.filterwarnings("ignore", module="IPython")
warnings.filterwarnings("ignore", category=DeprecationWarning)

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import numpy as np
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError, Timeout, RequestException
from collections import defaultdict
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata, visualization

today = datetime.today().date()

Project root dir: c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn


### 3.1.2. Load Settings and Initialize Hopsworks Connection 

In [56]:
# ---------------------------------------------------------
# 1. Detect environment (local, Jupyter, or Hopsworks Job)
# ---------------------------------------------------------
# Check for Hopsworks-specific environment variables
RUNNING_IN_HOPSWORKS = (
    "HOPSWORKS_PROJECT_ID" in os.environ or 
    "HOPSWORKS_JOB_ID" in os.environ or
    "HOPSWORKS_PROJECT_NAME" in os.environ
)

if RUNNING_IN_HOPSWORKS:
    print("🔧 Running in Hopsworks environment")
    # Running inside Hopsworks → use implicit login and secrets vault
    project = hopsworks.login()
    secrets_api = hopsworks.get_secrets_api()

    os.environ["HOPSWORKS_API_KEY"] = secrets_api.get_secret("HOPSWORKS_API_KEY").value
    os.environ["AQICN_API_KEY"] = secrets_api.get_secret("AQICN_API_KEY").value
    os.environ["GH_PAT"] = secrets_api.get_secret("GH_PAT").value
    os.environ["GH_USERNAME"] = secrets_api.get_secret("GH_USERNAME").value
    
    # Now load settings (will read from os.environ we just set)
    settings = config.HopsworksSettings()
    
    HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
    AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()
    GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

else:
    print("💻 Running locally")
    # Running locally → use .env file
    from dotenv import load_dotenv
    load_dotenv()
    
    # Load settings from .env
    settings = config.HopsworksSettings()
    
    HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
    AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()
    GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()
    
    # Login to Hopsworks using the API key
    project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)

# ---------------------------------------------------------
# 2. Get feature store (works for both environments now)
# ---------------------------------------------------------
fs = project.get_feature_store()


💻 Running locally
HopsworksSettings initialized!
2026-01-26 10:33:53,697 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-26 10:33:53,702 INFO: Initializing external client
2026-01-26 10:33:53,703 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-26 10:33:55,123 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


### 3.1.3. Repository Management

In [57]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

Already in repo at c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn


### 3.1.4. Configure API Keys and Secrets

In [ ]:
secrets = hopsworks.get_secrets_api()

try:
    secrets.get_secret("AQICN_API_KEY")
except:
    secrets.create_secret("AQICN_API_KEY", settings.AQICN_API_KEY.get_secret_value())

## 3.2. Get Feature Groups

In [59]:
air_quality_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

## 3.3. Load Sensor Locations from Feature Group

In [60]:
# Load data from air_quality feature group
aq_data = air_quality_fg.read()

if len(aq_data) == 0:
    print("⚠️ No air quality data found. Run pipeline 1 (backfill) first.")
    sys.exit(1)

# Extract metadata and set sensor_id as index for lookups
aq_metadata = aq_data[["sensor_id", "latitude", "longitude", "city", "street", "country"]].drop_duplicates(subset=["sensor_id"]).set_index("sensor_id")

# Build sensor location dictionary
sensor_locations = {}
for sensor_id, row in aq_metadata.iterrows():
    sensor_locations[sensor_id] = (
        row["latitude"], 
        row["longitude"], 
        row["city"], 
        row["street"], 
        row["country"]
    )
print(f"📍 Loaded locations for {len(sensor_locations)} sensors")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.35s) 
📍 Loaded locations for 103 sensors


## 3.4. Create Additional Feature Views
Create multiple feature views with different feature combinations (baseline, rolling windows, lagged features, nearby sensors, complete) for model comparison.

In [61]:
# Select features for training data
baseline_features = air_quality_fg.select(["pm25", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])

baseline_feature_view = fs.get_or_create_feature_view(
    name="air_quality_baseline_fv",
    description="Weather features for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=baseline_features,
)

rolling_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
rolling_feature_view = fs.get_or_create_feature_view(
    name="air_quality_rolling_fv",
    description="Weather features, PM2.5 rolling window (3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=rolling_features,
)

nearby_features = air_quality_fg.select(["pm25", "pm25_nearby_avg", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
nearby_feature_view = fs.get_or_create_feature_view(
    name="air_quality_nearby_fv",
    description="Weather features, PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=nearby_features,
)

lagged_1d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_1d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_1d_fv",
    description="Weather features, PM2.5 lags (1d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_1d_features,
)

lagged_2d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_2d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_2d_fv",
    description="Weather features, PM2.5 lags (1d, 2d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_2d_features,
)

lagged_3d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_3d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_3d_fv",
    description="Weather features, PM2.5 lags (1d, 2d, 3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_3d_features,
)

complete_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "pm25_nearby_avg", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
complete_feature_view = fs.get_or_create_feature_view(
    name="air_quality_complete_fv",
    description="Weather features, PM2.5 rolling window (3d), and PM2.5 lags (1d, 2d, 3d), and PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=complete_features,
)

2026-01-26 10:34:03,736 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-26 10:34:05,502 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-26 10:34:07,360 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-26 10:34:09,108 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-26 10:34

## 3.5. Model Training Setup

In [62]:
# Prepare containers to store models, predictions, and results
models = defaultdict(dict)
y_preds = defaultdict(dict)
results = []

# Define feature view dictionary for iteration
feature_views = {
    "baseline": baseline_feature_view,
    "rolling": rolling_feature_view,
    "nearby": nearby_feature_view,
    "lagged_1d": lagged_1d_feature_view,
    "lagged_2d": lagged_2d_feature_view,
    "lagged_3d": lagged_3d_feature_view,
    "complete": complete_feature_view,
}

## 3.7. Model Training Loop
Train XGBoost models for each feature combination and sensor, run 5 iterations per configuration, select best model based on R2 score, and store results.

### 3.7.1. Establish Directory

In [63]:
model_dir = f"{root_dir}/models"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    
print(f"Model directory: {model_dir}")

Model directory: c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models


### 3.7.2. Load and Cache Feature View Data

In [64]:
feature_data_cache = {}
total_views = len(feature_views)

for i, (feature_name, feature_view) in enumerate(feature_views.items(), start=1):
    print(f"Reading ({i}/{total_views}): {feature_name}")

    df = feature_view.query.read()
    df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)
    feature_data_cache[feature_name] = df

    print(f"    ✔ Loaded {len(df):,} rows")

print(f"\n✅ Loaded all {total_views} feature views")

Reading (1/7): baseline
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.96s) 
    ✔ Loaded 165,668 rows
Reading (2/7): rolling
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.87s) 
    ✔ Loaded 165,668 rows
Reading (3/7): nearby
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.66s) 
    ✔ Loaded 165,668 rows
Reading (4/7): lagged_1d
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.95s) 
    ✔ Loaded 165,668 rows
Reading (5/7): lagged_2d
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.69s) 
    ✔ Loaded 165,668 rows
Reading (6/7): lagged_3d
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.90s) 
    ✔ Loaded 165,668 rows
Reading (7/7): complete
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.86s) 
    ✔ Loaded 165,668 rows

✅ Loaded all 7 feature views


### 3.7.3. Build Task List

In [65]:
tasks = []
total_views = len(feature_data_cache)

print(f"Building task list from {total_views} feature views:")

for i, (feature_name, df) in enumerate(feature_data_cache.items(), start=1):
    sensor_ids = df["sensor_id"].unique()
    count = len(sensor_ids)

    print(f"[{i}/{total_views}] {feature_name}: {count} sensors")

    for sensor_id in sensor_ids:
        tasks.append((feature_name, sensor_id))

print(f"\n✅ {len(tasks):,} total training tasks")

Building task list from 7 feature views:
[1/7] baseline: 103 sensors
[2/7] rolling: 103 sensors
[3/7] nearby: 103 sensors
[4/7] lagged_1d: 103 sensors
[5/7] lagged_2d: 103 sensors
[6/7] lagged_3d: 103 sensors
[7/7] complete: 103 sensors

✅ 721 total training tasks


### 3.7.4. Initialize Training Containers

In [66]:
models = {name: {} for name in feature_views.keys()}
y_preds = {name: {} for name in feature_views.keys()}
results = []

print(f"✅ Initialized containers for {len(tasks)} training tasks")

✅ Initialized containers for 721 training tasks


### 3.7.5. Training Loop

In [67]:
total = len(tasks)

for idx, (feature_name, sensor_id) in enumerate(tasks, start=1):
    df = feature_data_cache[feature_name]
    df = df[df["sensor_id"] == sensor_id].dropna(subset=["pm25"])

    if len(df) < 10:
        continue

    feature_cols = [c for c in df.columns if c not in [
        "pm25","date","sensor_id","city","street","country",
        "latitude","longitude","aqicn_url"
    ]]

    train_size = int(0.8 * len(df))
    train_df = df.iloc[:train_size]
    test_df = df.iloc[train_size:]

    if len(test_df) < 2:
        continue

    X_train = train_df[feature_cols]
    y_train = train_df["pm25"]
    X_test = test_df[feature_cols]
    y_test = test_df["pm25"]

    best_model = None
    best_r2 = -1e9
    best_mse = 1e9

    for i in range(5):
        model = XGBRegressor(
            n_estimators=100,
            learning_rate=0.05,
            random_state=165439*i
        )
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        r2 = r2_score(y_test, pred)
        mse = mean_squared_error(y_test, pred)

        if r2 > best_r2:
            best_r2 = r2
            best_mse = mse
            best_model = model

    # Store results
    models.setdefault(feature_name, {})[sensor_id] = best_model
    y_preds.setdefault(feature_name, {})[sensor_id] = best_model.predict(X_test)
    results.append({
        "feature_name": feature_name,
        "sensor_id": sensor_id,
        "R2": best_r2,
        "MSE": best_mse,
        "train_size": len(train_df),
        "test_size": len(test_df),
    })

    if idx % 10 == 0 or idx == total:
        print(f"[{idx}/{total}] Trained {feature_name} / sensor {sensor_id}: R²={best_r2:.3f}, MSE={best_mse:.2f}")

print(f"\n✅ Training complete: {len(results)} models trained")

[10/721] Trained baseline / sensor 59593: R²=0.031, MSE=5.16
[20/721] Trained baseline / sensor 60838: R²=0.041, MSE=97.27
[30/721] Trained baseline / sensor 62566: R²=0.147, MSE=10.94
[40/721] Trained baseline / sensor 65707: R²=0.020, MSE=15.46
[50/721] Trained baseline / sensor 79750: R²=0.055, MSE=4.62
[60/721] Trained baseline / sensor 89584: R²=-0.032, MSE=18.37
[70/721] Trained baseline / sensor 122302: R²=-0.107, MSE=6.78
[80/721] Trained baseline / sensor 196735: R²=-0.069, MSE=3.66
[90/721] Trained baseline / sensor 404209: R²=-0.005, MSE=9.26
[100/721] Trained baseline / sensor 497266: R²=0.360, MSE=4.81
[110/721] Trained rolling / sensor 68167: R²=-2.417, MSE=683.33
[120/721] Trained rolling / sensor 61861: R²=0.184, MSE=3.79
[130/721] Trained rolling / sensor 65104: R²=0.412, MSE=0.49
[140/721] Trained rolling / sensor 58921: R²=0.897, MSE=0.04
[150/721] Trained rolling / sensor 88876: R²=0.227, MSE=11.58
[160/721] Trained rolling / sensor 59650: R²=0.036, MSE=8.31
[170/72

In [68]:
# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     data['date'] = pd.to_datetime(data['date']).dt.tz_localize(None)

#     for sensor_id in data["sensor_id"].unique():
#         df = data[data["sensor_id"] == sensor_id].copy()

#         # Drop rows where pm25 is NaN
#         df = df.dropna(subset=["pm25"])
        
#         if len(df) < 10:
#             print(f"⚠️  Skipping sensor {sensor_id} ({feature_name}): insufficient data ({len(df)} rows)")
#             continue

#         # Identify which features this feature view actually uses
#         # Drop non-feature columns first
#         feature_cols = [c for c in df.columns if c not in ["pm25", "date", "sensor_id", "city", "street", "country", "latitude", "longitude", "aqicn_url"]]
        
#         # Keep rows with NaN in features - XGBoost handles NaN natively
#         df_clean = df.copy()
        
#         if len(df_clean) < 10:
#             print(f"⚠️  Skipping sensor {sensor_id} ({feature_name}): insufficient clean data ({len(df_clean)} rows, needed features: {feature_cols})")
#             continue

#         # Split
#         train_size = int(0.8 * len(df_clean))
#         train_df = df_clean.iloc[:train_size]
#         test_df = df_clean.iloc[train_size:]

#         if len(test_df) < 2:
#             print(f"⚠️  Skipping sensor {sensor_id} ({feature_name}): test set too small ({len(test_df)} rows)")
#             continue

#         # Prepare features and target
#         X_train = train_df[feature_cols]
#         y_train = train_df["pm25"]
#         X_test = test_df[feature_cols]
#         y_test = test_df["pm25"]

#         best_r2 = -float('inf')
#         best_mse = float('inf')
#         best_model = None

#         mse_list = []
#         r2_list = []
        
#         for i in range(5):
#             model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=165439*i)
#             model.fit(X_train, y_train)
#             y_pred = model.predict(X_test)
#             r2 = r2_score(y_test, y_pred)
#             mse = mean_squared_error(y_test, y_pred)
#             r2_list.append(r2)
#             mse_list.append(mse)
            
#             if r2 > best_r2:
#                 best_r2 = r2
#                 best_mse = mse
#                 best_model = model

#         models[feature_name][sensor_id] = best_model

#         if best_model is not None:
#             y_preds[feature_name][sensor_id] = best_model.predict(X_test)
#             results.append({
#                 "feature_name": feature_name,
#                 "sensor_id": sensor_id,
#                 "MSE": sum(mse_list) / len(mse_list),
#                 "R2": sum(r2_list) / len(r2_list),
#                 "train_size": len(X_train),
#                 "test_size": len(X_test),
#             })
#             print(f"✅ {feature_name} - Sensor {sensor_id}: R2={best_r2:.3f}, MSE={best_mse:.2f}")
#         else:
#             print(f"⚠️  No valid model for {feature_name} - {sensor_id}")

In [69]:
# for feature_name, feature_view in feature_views.items():
#     data = feature_view.query.read()
#     data['date'] = pd.to_datetime(data['date']).dt.tz_localize(None)

#     for sensor_id in metadata_df.index:
#         df = data[data["sensor_id"] == sensor_id].copy()

#         # Clean the data before splitting
#         df = df.dropna(subset=["pm25"])
#         # features_for_cleaning = df.drop(columns=["pm25", "date", "city", "sensor_id"])
#         features_for_cleaning = df.drop(columns=["pm25", "date", "sensor_id"])

#         target_for_cleaning = df["pm25"]
#         clean_mask = ~(features_for_cleaning.isna().any(axis=1) | target_for_cleaning.isna())
#         df_clean = df[clean_mask].copy()
#         if len(df_clean) < 10:
#             print(f"⚠️  Skipping sensor {sensor_id}: insufficient data after cleaning ({len(df_clean)} rows)")
#             continue

#         train_size = int(0.8 * len(df_clean))
#         train_df = df_clean.iloc[:train_size]
#         test_df = df_clean.iloc[train_size:]

#         if len(test_df) < 2:
#             print(f"⚠️  Skipping sensor {sensor_id}: test set too small after split ({len(test_df)} rows)")
#             continue

#         # Drop non-feature columns (pm25 is target, others are metadata)
#         X_train = train_df.drop(columns=["pm25", "date", "city", "sensor_id"])
#         y_train = train_df["pm25"]
#         X_test = test_df.drop(columns=["pm25", "date", "city", "sensor_id"])
#         y_test = test_df["pm25"]

#         best_r2 = -float('inf')
#         best_mse = float('inf')
#         best_model = None

#         mse_list = []
#         r2_list = []
#         for i in range(5):
#             model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=165439*i)
#             model.fit(X_train, y_train)
#             y_pred = model.predict(X_test)
#             r2 = r2_score(y_test, y_pred)
#             mse = mean_squared_error(y_test, y_pred)
#             r2_list.append(r2)
#             mse_list.append(mse)
#             if r2 > best_r2:
#                 best_r2 = r2
#                 best_mse = mse
#                 best_model = model

#         models[feature_name][sensor_id] = best_model

#         if best_model is not None:
#             y_preds[feature_name][sensor_id] = best_model.predict(X_test)
#             results.append({
#                 "feature_name": feature_name,
#                 "sensor_id": sensor_id,
#                 "MSE": sum(mse_list) / len(mse_list),
#                 "R2": sum(r2_list) / len(r2_list),
#                 "train_size": len(X_train),
#                 "test_size": len(X_test),
#             })
#         else:
#             print(f"⚠️  No valid model trained for {feature_name} - {sensor_id}, R2 scores: {r2_list}, Best R2: {best_r2}")
            

## 3.9. Model Selection and Saving
Identify best performing model for each sensor, save models and feature importance plots, and prepare test data with predictions.

### 3.9.1. Find Best Model per Sensor

In [70]:
results_df = pd.DataFrame(results)

best_models = results_df.loc[results_df.groupby("sensor_id")["R2"].idxmax()]
best_models = best_models.set_index("sensor_id")

print(f"Identified best models for {len(best_models)} sensors:\n")

summary = best_models[["feature_name", "R2", "MSE"]].sort_index()

print(summary.to_string())


Identified best models for 103 sensors:

          feature_name        R2         MSE
sensor_id                                   
57421        lagged_1d  0.419225    5.296772
58666        lagged_1d  0.157236    5.798248
58909        lagged_1d  0.374371    4.101138
58912          rolling  0.430939    5.724496
58921         complete  0.919423    0.043539
59095        lagged_3d  0.318437    9.876509
59356        lagged_1d  0.364699   11.054319
59410        lagged_3d  0.278112    0.249105
59497        lagged_1d  0.368901    5.938145
59593        lagged_3d  0.218126    4.166982
59650        lagged_1d  0.345838    6.139885
59656        lagged_3d  0.058061   68.517111
59887        lagged_1d  0.238314   10.359162
59893          rolling  0.535112    1.346819
59899        lagged_1d  0.364168    7.452584
60073        lagged_2d  0.466981    2.714573
60076        lagged_1d  0.414258   13.915928
60535        lagged_1d  0.198454   17.757584
60541         complete  0.297254   11.286639
60838        l

### 3.9.2. Load Feature View Data

In [71]:
total_views = len(feature_views)
print(f"Loading {total_views} feature views into memory...\n")

cached_feature_data = {}

for i, (name, fv) in enumerate(feature_views.items(), start=1):
    print(f"[{i}/{total_views}] Reading feature view: {name}...")
    df_cached = fv.query.read()
    cached_feature_data[name] = df_cached
    print(f"    ✔ Loaded {len(df_cached):,} rows")

print("\nNormalizing date columns...\n")

for name, df_cached in cached_feature_data.items():
    df_cached["date"] = pd.to_datetime(df_cached["date"]).dt.tz_localize(None)
    print(f"    ✔ Normalized dates for '{name}'")

print("\nAll feature views loaded and normalized")


Loading 7 feature views into memory...

[1/7] Reading feature view: baseline...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.30s) 
    ✔ Loaded 165,668 rows
[2/7] Reading feature view: rolling...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.53s) 
    ✔ Loaded 165,668 rows
[3/7] Reading feature view: nearby...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.25s) 
    ✔ Loaded 165,668 rows
[4/7] Reading feature view: lagged_1d...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.46s) 
    ✔ Loaded 165,668 rows
[5/7] Reading feature view: lagged_2d...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.83s) 
    ✔ Loaded 165,668 rows
[6/7] Reading feature view: lagged_3d...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.17s) 
    ✔ Loaded 165,668 rows
[7/7] Reading feature view: complete...
Finished

### 3.9.3. Generate Predictions

In [72]:
all_test_data = []
total_sensors = len(best_models)

print(f"Processing {total_sensors} sensors...\n")

for idx, (_, row) in enumerate(best_models.iterrows(), start=1):
    sensor_id = row.name
    best_feature = row["feature_name"]

    status = []  # collect short status flags

    # --- Save model + feature importance ---
    sensor_dir = f"{model_dir}/{sensor_id}"
    images_dir = f"{sensor_dir}/images"
    os.makedirs(images_dir, exist_ok=True)

    best_model = models[best_feature][sensor_id]
    best_model.save_model(f"{sensor_dir}/model.json")
    plot_importance(best_model)
    plt.savefig(f"{images_dir}/feature_importance.png")
    plt.close()
    status.append("model+plot")

    # --- Load cached feature view data ---
    df = cached_feature_data[best_feature]
    df = df[df["sensor_id"] == sensor_id].copy()
    df = df.sort_values("date").reset_index(drop=True)
    df = df.dropna(subset=["pm25"])

    feature_cols = [
        c for c in df.columns
        if c not in ["pm25", "date", "sensor_id", "city", "street",
                     "country", "latitude", "longitude", "aqicn_url"]
    ]

    df_clean = df.copy()
    if len(df_clean) < 10:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (too few rows)")
        continue

    # --- Full predictions ---
    X_full = df_clean[feature_cols]
    df_clean["predicted_pm25"] = best_model.predict(X_full)
    df_clean["best_model"] = best_feature
    status.append("predictions")

    # --- Hindcast window ---
    cutoff_date = pd.Timestamp.now() - pd.DateOffset(months=18)
    df_hindcast = df_clean[df_clean["date"] >= cutoff_date].copy()

    if len(df_hindcast) == 0:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no recent data)")
        continue
    status.append("hindcast")
  
    # --- Metadata ---
    if sensor_id in aq_metadata.index:
        df_clean["latitude"] = aq_metadata.at[sensor_id, "latitude"]
        df_clean["longitude"] = aq_metadata.at[sensor_id, "longitude"]
        df_clean["city"] = aq_metadata.at[sensor_id, "city"]
        df_clean["street"] = aq_metadata.at[sensor_id, "street"]
        df_clean["sensor_id"] = sensor_id
        status.append("metadata")
    else:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no metadata)")
        continue

    # --- Append final data ---
    all_test_data.append(
        df_clean[[
            "date", "sensor_id", "pm25", "predicted_pm25",
            "latitude", "longitude", "city", "street", "best_model"
        ]]
    )
    status.append("saved")

    # --- One-line summary ---
    print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: " + ", ".join(status))

print(f"\nDone. Successfully processed {len(all_test_data)} sensors.")

Processing 103 sensors...

[1/103] Sensor 57421: model+plot, predictions, hindcast, metadata, saved
[2/103] Sensor 58666: model+plot, predictions, hindcast, metadata, saved
[3/103] Sensor 58909: model+plot, predictions, hindcast, metadata, saved
[4/103] Sensor 58912: skipped (no recent data)
[5/103] Sensor 58921: model+plot, predictions, hindcast, metadata, saved
[6/103] Sensor 59095: model+plot, predictions, hindcast, metadata, saved
[7/103] Sensor 59356: model+plot, predictions, hindcast, metadata, saved
[8/103] Sensor 59410: model+plot, predictions, hindcast, metadata, saved
[9/103] Sensor 59497: model+plot, predictions, hindcast, metadata, saved
[10/103] Sensor 59593: model+plot, predictions, hindcast, metadata, saved
[11/103] Sensor 59650: model+plot, predictions, hindcast, metadata, saved
[12/103] Sensor 59656: model+plot, predictions, hindcast, metadata, saved
[13/103] Sensor 59887: model+plot, predictions, hindcast, metadata, saved
[14/103] Sensor 59893: model+plot, predictions

In [73]:
all_test_data = []

# --- Loop through best models ---
for _, row in best_models.iterrows():
    sensor_id = row.name
    print(sensor_id)

    best_feature = row["feature_name"]

    # --- Save model + feature importance ---
    sensor_dir = f"{model_dir}/{sensor_id}"
    images_dir = f"{sensor_dir}/images"
    os.makedirs(images_dir, exist_ok=True)

    best_model = models[best_feature][sensor_id]
    best_model.save_model(f"{sensor_dir}/model.json")

    plot_importance(best_model)
    plt.savefig(f"{images_dir}/feature_importance.png")
    plt.close()

    # --- Load cached feature view data ---
    df = cached_feature_data[best_feature]
    df = df[df["sensor_id"] == sensor_id].copy()

    # --- EXACT same cleaning as training ---
    df = df.sort_values("date").reset_index(drop=True)
    df = df.dropna(subset=["pm25"])

    feature_cols = [
        c for c in df.columns
        if c not in ["pm25", "date", "sensor_id", "city", "street", "country", "latitude", "longitude", "aqicn_url"]
    ]

    # Keep rows with NaN in features - same as training (XGBoost handles NaN)
    df_clean = df.copy()
    if len(df_clean) < 10:
        continue

    # --- Generate predictions for FULL dataset ---
    X_full = df_clean[feature_cols]
    predictions_full = best_model.predict(X_full)
    df_clean["predicted_pm25"] = predictions_full
    df_clean["best_model"] = best_feature
    
    # --- Filter to last 18 months for hindcast visualization ---
    cutoff_date = pd.Timestamp.now() - pd.DateOffset(months=18)
    df_hindcast = df_clean[df_clean["date"] >= cutoff_date].copy()
    
    if len(df_hindcast) == 0:
        print(f"⚠️ Skipping sensor {sensor_id}: no data in last 18 months")
        continue

   # Get lat/lon AND city/street from metadata
    if sensor_id in aq_data.index:
        df_clean["latitude"] = aq_data.at[sensor_id, "latitude"]
        df_clean["longitude"] = aq_data.at[sensor_id, "longitude"]
        df_clean["city"] = aq_data.at[sensor_id, "city"]
        df_clean["street"] = aq_data.at[sensor_id, "street"]
        df_clean["sensor_id"] = sensor_id
    else:
        print(f"⚠️ Skipping sensor {sensor_id}: metadata not found")
        continue

    all_test_data.append(
        df_clean[["date", "sensor_id", "pm25", "predicted_pm25", "latitude", "longitude", "city", "street", "best_model"]]
    )

57421
58666
58909
58912
⚠️ Skipping sensor 58912: no data in last 18 months
58921
59095
59356
59410
59497
59593
59650
59656
59887
59893
59899
60073
⚠️ Skipping sensor 60073: no data in last 18 months
60076
60535
60541
60838
60853
60859
60886
60889
⚠️ Skipping sensor 60889: no data in last 18 months
61045
61420
61714
61861
61867
62566
62848
62968
⚠️ Skipping sensor 62968: no data in last 18 months
63637
63646
65104
65146
65272
65284
65290
65707
68167
69628
69724
70564
76915
77446
77488
78529
78532
79750
79999
80773
81505
82384
82942
84085
87319
88372
88876
89584
90676
92683
105325
107110
112672
112993
113539
113542
⚠️ Skipping sensor 113542: no data in last 18 months
121810
122302
128095
129124
149242
154549
163156
180187
⚠️ Skipping sensor 180187: metadata not found
191047
⚠️ Skipping sensor 191047: metadata not found
192520
⚠️ Skipping sensor 192520: metadata not found
194215
⚠️ Skipping sensor 194215: metadata not found
196735
⚠️ Skipping sensor 196735: metadata not found
198559
⚠️ S

In [74]:
# --- DIAGNOSTIC: Check data coverage by year ---
print("\n📊 DATA COVERAGE DIAGNOSTIC")
print("="*60)

for feature_name, df_cached in cached_feature_data.items():
    print(f"\n{feature_name}:")
    
    # Check one sensor as example
    sample_sensor = df_cached["sensor_id"].iloc[0]
    sensor_data = df_cached[df_cached["sensor_id"] == sample_sensor].copy()
    sensor_data["year"] = pd.to_datetime(sensor_data["date"]).dt.year
    
    print(f"  Sensor {sample_sensor}:")
    print(f"  Total rows: {len(sensor_data)}")
    print(f"  Date range: {sensor_data['date'].min()} to {sensor_data['date'].max()}")
    
    # Check pm25 by year
    yearly_stats = sensor_data.groupby("year").agg({
        "pm25": ["count", lambda x: x.isna().sum()]
    })
    yearly_stats.columns = ["Total Rows", "NaN Count"]
    yearly_stats["Non-NaN"] = yearly_stats["Total Rows"] - yearly_stats["NaN Count"]
    print("\n  PM2.5 availability by year:")
    print(yearly_stats)
    
    # After dropna
    sensor_data_clean = sensor_data.dropna(subset=["pm25"])
    if len(sensor_data_clean) > 0:
        print(f"\n  After dropna(subset=['pm25']):")
        print(f"    Remaining rows: {len(sensor_data_clean)}")
        print(f"    Date range: {sensor_data_clean['date'].min()} to {sensor_data_clean['date'].max()}")
        print(f"    Years present: {sorted(sensor_data_clean['year'].unique())}")
    else:
        print(f"\n  ⚠️ NO DATA LEFT after dropna!")
    
    break  # Only check first feature view for now

print("\n" + "="*60)


📊 DATA COVERAGE DIAGNOSTIC

baseline:
  Sensor 57421:
  Total rows: 2062
  Date range: 2019-12-09 00:00:00 to 2026-01-14 00:00:00

  PM2.5 availability by year:
      Total Rows  NaN Count  Non-NaN
year                                
2019          23          0       23
2020         359          0      359
2021         267          0      267
2022         363          0      363
2023         365          0      365
2024         347          0      347
2025         334          0      334
2026           4          0        4

  After dropna(subset=['pm25']):
    Remaining rows: 2062
    Date range: 2019-12-09 00:00:00 to 2026-01-14 00:00:00
    Years present: [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]



In [75]:
# # Find best model (highest R2) for each sensor
# results_df = pd.DataFrame(results)
# best_models = results_df.loc[results_df.groupby('sensor_id')['R2'].idxmax()]
# best_models = results_df.loc[results_df.groupby('sensor_id')['R2'].idxmax()]

# print("Best models per sensor:")
# print(best_models[['sensor_id', 'feature_name', 'R2', 'MSE']])

# all_data = baseline_features.read()
# all_data['date'] = pd.to_datetime(all_data['date']).dt.tz_localize(None)

# cached_feature_data = {
#     name: fv.query.read()
#     for name, fv in feature_views.items()
# }

# all_test_data = []

# for _, row in best_models.iterrows():
#     sensor_id = row['sensor_id']
#     best_feature = row['feature_name']
    
#     sensor_dir = f"{model_dir}/{sensor_id}"
#     if not os.path.exists(sensor_dir):
#         os.mkdir(sensor_dir)
#     images_dir = f"{model_dir}/{sensor_id}/images"
#     if not os.path.exists(images_dir):
#         os.mkdir(images_dir)

#     best_model = models[best_feature][sensor_id]
#     model_path = f"{sensor_dir}/model.json"
#     plot_importance(best_model)
#     importance_path = f"{images_dir}/feature_importance.png"
#     plt.savefig(importance_path)
#     plt.close()
    
#     best_model.save_model(model_path)

#     # Use the same feature view and data processing logic that was used for training
#     best_feature_view = feature_views[best_feature]
#     sensor_data = cached_feature_data[best_feature]
#     sensor_data['date'] = pd.to_datetime(sensor_data['date']).dt.tz_localize(None)
    
#     df = sensor_data[sensor_data['sensor_id'] == sensor_id].copy()
    
#     df = df.sort_values("date").reset_index(drop=True)

#     # Apply EXACT same cleaning logic as in training loop
#     df = df.dropna(subset=['pm25'])
    
#     # Recompute feature columns exactly as in training
#     feature_cols = [c for c in df.columns if c not in ["pm25", "date", "sensor_id", "location_id"]]

#     # Drop rows with NaN in any feature
#     df_clean = df.dropna(subset=feature_cols).copy()

#     if len(df_clean) < 10:
#         continue

#     # Same split as training
#     train_size = int(0.8 * len(df_clean))
#     test_df = df_clean.iloc[train_size:].copy()

#     # # Create feature matrix for comprehensive NaN cleaning (same as training)
#     # features_for_cleaning = df.drop(columns=["pm25", "date", "sensor_id"])
#     # target_for_cleaning = df["pm25"]
    
#     # # Remove rows with NaN values in any feature or target (same as training)
#     # clean_mask = ~(features_for_cleaning.isna().any(axis=1) | target_for_cleaning.isna())
#     # df_clean = df[clean_mask].copy()
    
#     # if len(df_clean) < 10:
#     #     continue
        
#     # # Split the cleaned data (same as training)
#     # train_size = int(0.8 * len(df_clean))
#     # test_df = df_clean.iloc[train_size:].copy()
    
#     if len(test_df) == 0:
#         continue
    
#     # Test data is already clean from the comprehensive cleaning above
#     clean_test_df = test_df.copy()
#     predictions = y_preds[best_feature][sensor_id]
    
#     if len(clean_test_df) == len(predictions):
#         clean_test_df['predicted_pm25'] = predictions
#         clean_test_df['best_model'] = best_feature
#         all_test_data.append(clean_test_df[['date', 'pm25', 'predicted_pm25', 'latitude', 'longitude', 'best_model']])
#     else:
#         print(f"⚠️  Skipping sensor {sensor_id}: prediction length mismatch ({len(predictions)} vs {len(clean_test_df)})")

## 3.10. Model Registration & Visualization
Create prediction plots for each sensor, register model in Hopsworks model registry with metrics and save models with their configuration.

### 3.10.1. Setup

In [76]:
print("Columns in df:", df.columns.tolist())
print("df shape:", df.shape)

Columns in df: ['pm25', 'pm25_lag_1d', 'date', 'sensor_id', 'temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant']
df shape: (86, 8)


In [77]:
df = pd.concat(all_test_data, ignore_index=True) if all_test_data else pd.DataFrame()
df = df.sort_values(by=["date"])
df_by_sensor = {sid: g.copy() for sid, g in df.groupby("sensor_id")}

### 3.10.2. Precompute Best Model per Sensor

In [78]:
# best_model_per_sensor = (
#     results_df.sort_values("R2", ascending=False)
#               .groupby("sensor_id")
#               .first()[["feature_name", "R2", "MSE"]]
#               .rename(columns={"feature_name": "best_model"})
# )

### 3.10.3. Create Training Datasets

In [79]:
training_versions = {}

for model_name, fv in feature_views.items():
    print(f"\nCreating training dataset for {model_name}...")
    retries = 5  # Increased from 3 to handle server overload
    delay = 10   # Increased from 5 seconds to give server more recovery time
    
    for attempt in range(1, retries + 1):
        try:
            td_version, td_job = fv.create_training_data(
                description=f"Training data for {model_name}",
                data_format="csv",
                write_options={"wait_for_job": True}
            )
            
            version = td_version.version if hasattr(td_version, "version") else td_version
            training_versions[model_name] = version
            print(f"✔ Created training dataset for {model_name}: version {version}")
            break  # Success - exit retry loop
        
        except Exception as e:
            if attempt < retries:
                print(f"⚠️ Attempt {attempt}/{retries} failed for {model_name}: {e}. Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"❌ All {retries} attempts failed for {model_name}: {e}")
                training_versions[model_name] = None

print(f"\n✅ Created {len([v for v in training_versions.values() if v is not None])} training datasets")


Creating training dataset for baseline...
Training dataset job started successfully, you can follow the progress at 
http://c.app.hopsworks.ai/p/1279184/jobs/named/air_quality_baseline_fv_1_create_fv_td_26012026094637/executions
2026-01-26 10:46:47,326 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2026-01-26 10:46:50,495 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-26 10:46:53,672 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-26 10:51:48,071 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-26 10:51:48,218 INFO: Waiting for log aggregation to finish.
2026-01-26 10:51:56,781 INFO: Execution finished successfully.
2026-01-26 10:51:56,964 WARNING: VersionWarning: Incremented version to `21`.

✔ Created training dataset for baseline: version 21

Creating training dataset for rolling...
Train

### 3.10.4. Register Models

In [ ]:
uploaded = 0
total_sensors = len(df_by_sensor)

print(f"Registering models for {total_sensors} sensors...\n")

for sensor_id in df_by_sensor.keys():  # Loop through sensors that have predictions
    df_subset = df_by_sensor[sensor_id].copy()
    
    city = df_subset['city'].iloc[0]
    street = df_subset['street'].iloc[0]

    if sensor_id not in best_models.index:
        print(f"[skip] Sensor {sensor_id}: no best model found")
        continue

    best_model_name = best_models.loc[sensor_id, "feature_name"]
    best_model_r2 = best_models.loc[sensor_id, "R2"]
    best_model_mse = best_models.loc[sensor_id, "MSE"]

    best_model_feature_view = feature_views[best_model_name]
    training_dataset_version = training_versions.get(best_model_name, None)

    # Drop unnecessary columns for visualization
    df_subset = df_subset.drop(
        columns=["latitude", "longitude", "best_model", "sensor_id", "city", "street"], 
        errors="ignore"
    )

    images_dir = f"{model_dir}/{sensor_id}/images"
    os.makedirs(images_dir, exist_ok=True)
    image_path = f"{images_dir}/hindcast_training.png"

    fig = visualization.plot_air_quality_forecast(
        city, street, df_subset, image_path, hindcast=True
    )
    if fig is not None:
        fig.suptitle(f"{city} {street} (Best Model: {best_model_name})")
        plt.close(fig)

    # Get the trained XGBoost model
    xgb_model = models[best_model_name][sensor_id]
    
    # Save model locally first (already done in 3.9.3, but ensure it's there)
    sensor_model_dir = f"{model_dir}/{sensor_id}"
    os.makedirs(sensor_model_dir, exist_ok=True)
    xgb_model.save_model(f"{sensor_model_dir}/model.json")

    # Register model
    model_kwargs = {
        "name": f"air_quality_xgboost_model_{sensor_id}",
        "metrics": {
            "R2": best_model_r2,
            "MSE": best_model_mse,
        },
        "feature_view": best_model_feature_view,
        "description": (
            f"Air Quality (PM2.5) predictor for {city} {street} "
            f"using {best_model_name} configuration"
        ),
    }

    if training_dataset_version is not None:
        model_kwargs["training_dataset_version"] = training_dataset_version

    mr = project.get_model_registry()
    aq_model = mr.python.create_model(**model_kwargs)
    aq_model.save(sensor_model_dir)
    
    uploaded += 1
    print(f"[{uploaded}/{total_sensors}] Sensor {sensor_id}: registered ({best_model_name})")

print(f"✅ {uploaded} models registered and visualizations generated.")


Registering models for 70 sensors...



  0%|          | 0/6 [00:00<?, ?it/s]

Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/57421/model.json…

Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/57421\images/fea…

Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/57421\images/hin…

...

Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/163156/model.jso…

Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/163156\images/fe…

Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/163156\images/hi…

Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn\model_schema.json: 0.00…

Model created, explore it at https://c.app.hopsworks.ai:443/p/1279184/models/air_quality_xgboost_model_163156/19
[70/70] Sensor 163156: registered (lagged_3d)
✅ 70 models registered and visualizations generated.


In [81]:

# df = pd.concat(all_test_data, ignore_index=True) if all_test_data else pd.DataFrame()
# df = df.sort_values(by=["date"])

# # Plot the best model for each sensor
# for sensor_id, meta in metadata_df.iterrows():
#     city = meta["city"]
#     street = meta["street"]
#     latitude = meta["latitude"]
#     longitude = meta["longitude"]
    
#     df_subset = df[(df["latitude"] == latitude) & (df["longitude"] == longitude)].copy()
#     if len(df_subset) == 0:
#         continue
    
#     # Get the best model name for display
#     best_model_name = df_subset['best_model'].iloc[0] if 'best_model' in df_subset.columns else 'unknown'
#     best_model_r2 = df_subset['R2'].iloc[0] if 'R2' in df_subset.columns else 0
#     best_model_mse = df_subset['MSE'].iloc[0] if 'MSE' in df_subset.columns else 0
#     best_model_feature_view = feature_views[best_model_name]
    
#     df_subset = df_subset.sort_values(by=["date"])
#     df_subset = df_subset.drop(columns=["latitude", "longitude", "best_model"])
    
#     images_dir = f"{model_dir}/{sensor_id}/images"
#     image_path = f"{images_dir}/hindcast_training.png"
    
#     fig = visualization.plot_air_quality_forecast(
#         city, street, df_subset, image_path, hindcast=True
#     )
#     if fig is not None:
#         fig.suptitle(f"{city} {street} (Best Model: {best_model_name})")
#         plt.close(fig)  # Clean up after saving

#     aq_model = mr.python.create_model(
#         name=f"air_quality_xgboost_model_{sensor_id}",
#         metrics={
#             "R2": best_model_r2,
#             "MSE": best_model_mse,
#         },
#         feature_view=best_model_feature_view,
#         description=f"Air Quality (PM2.5) predictor for {city} {street} using {best_model_name} configuration",
#     )

#     aq_model.save(f"{model_dir}/{sensor_id}")

## 3.11. Upload Plots

In [82]:
dataset_api = project.get_dataset_api()
base_dir = "Resources/plots"
try:
    dataset_api.mkdir(base_dir)
except:
    pass

uploaded_images = 0
total_sensors = len(sensor_locations)

print(f"Uploading plots for {total_sensors} sensors...\n")

for sensor_id in sensor_locations.keys():
    sensor_dir = f"{base_dir}/{sensor_id}"
    try:
        dataset_api.mkdir(sensor_dir)
    except:
        pass

    local_path = f"{model_dir}/{sensor_id}/images/hindcast_training.png"
    remote_path = f"{sensor_dir}/hindcast_training.png"

    ok = hopsworks_admin.safe_upload(dataset_api, local_path, remote_path)

    if ok:
        uploaded_images += 1
        print(f"Uploaded image for sensor {sensor_id} ({uploaded_images}/{total_sensors})")
    else:
        print(f"❌ [fail] Sensor {sensor_id}: upload failed after retries")

print(f"✅ Done uploading {uploaded_images} images.")


Uploading plots for 103 sensors...



Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/60853/images/hin…

Uploaded image for sensor 60853 (1/103)


Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/59497/images/hin…

Uploaded image for sensor 59497 (2/103)


...

Uploaded image for sensor 154549 (66/103)


Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/61420/images/hin…

Uploaded image for sensor 61420 (67/103)
⚠️ Upload failed (1/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\\notebooks\\pm25-forecast-openmeteo-aqicn/models/404209/images/hindcast_training.png'
⚠️ Upload failed (2/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\\notebooks\\pm25-forecast-openmeteo-aqicn/models/404209/images/hindcast_training.png'
⚠️ Upload failed (3/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\\notebooks\\pm25-forecast-openmeteo-aqicn/models/404209/images/hindcast_training.png'
⚠️ Upload failed (4/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\\notebooks\\pm25-forecast-openmeteo-aqicn/models/404209/images/hindcast_training.png'
⚠️ Upload failed (5/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\

Uploading c:\Users\krist\Documents\GitHub\pm25\notebooks\pm25-forecast-openmeteo-aqicn/models/59899/images/hin…

Uploaded image for sensor 59899 (68/103)
⚠️ Upload failed (1/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\\notebooks\\pm25-forecast-openmeteo-aqicn/models/533086/images/hindcast_training.png'
⚠️ Upload failed (2/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\\notebooks\\pm25-forecast-openmeteo-aqicn/models/533086/images/hindcast_training.png'
⚠️ Upload failed (3/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\\notebooks\\pm25-forecast-openmeteo-aqicn/models/533086/images/hindcast_training.png'
⚠️ Upload failed (4/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\\notebooks\\pm25-forecast-openmeteo-aqicn/models/533086/images/hindcast_training.png'
⚠️ Upload failed (5/5): [WinError 2] The system cannot find the file specified: 'c:\\Users\\krist\\Documents\\GitHub\\pm25\